In this exercise, we are going to fine-tune a model on a supervised **relation extraction** dataset.

The goal of the model is to predict, given a sentence and the character spans of two entities within the sentence, the relationship between the entities.

For example given the sentence:


**John, who played last night, is Doe's father.**


The model given the sentence and the spans of the entities John a Doe the model will have to predict what is the relation between Dohn and Doe from a set of pre-defined relations in this case the relation is parents (please note that some of the relations are one-way relations)

The dataset we will use is a subset of the [TACRED](https://nlp.stanford.edu/projects/tacred/) dataset, a supervised relation extraction dataset by Stanford University. 


As you realize by now, the straight forward supervised approach is just to take one of the transformers and use the sentence and the entity spans as input. However, in this exercise we will try a different approach using **Question Answering (QA)**.

Instead of just using the entities span and the sentence, we will train a model to answer the following questions "Who are the parents of John?","Who are the children of Doe?". If the question answering model will be able to answer the question succusfully than we will be able to conclude that the relation between the two entites exists.

In general **for each realtion** we will need to come up with template questions: In the example above the template questions corresponding to the parents relation are: 

*   Who are the parents of E1?
*   Who are the children of E2?

where E1 and E2 are the entities.



# Your part

You are required to fine-tune a model for relation extraction using the question answering framework.

Notes:


* In previous lectures we have seen a demo notebook demonstrating how to fine-tune a transformer on SQUAD, **from a technical prespective we are doing the same.**
*   For each one of the seven relations in the dataset you will need to find appropriate questions (please note that the questions must be SQUAD-like, meaning that if the answer exists it must be contained within the sentence in a contiguous way.)
* There are several issues that you will need to consider. Please provide a brief explanation whenever you tackle such issues


## Data

Let's download the data from the web, hosted on Dropbox.

In [ ]:
!pip install -q transformers 

     |████████████████████████████████| 2.5MB 24.6MB/s 
     |████████████████████████████████| 901kB 50.5MB/s 
     |████████████████████████████████| 3.3MB 55.8MB/s 


In [ ]:
import requests, zipfile, io

def download_data():
    url = "https://www.dropbox.com/s/izi2x4sjohpzoot/relation_extraction_dataset.zip?dl=1"
    r = requests.get(url)
    z = zipfile.ZipFile(io.BytesIO(r.content))
    z.extractall()

download_data()

Each row in the dataframe consists of a news article, and a sentence in which a certain relationship was found (just as "invested_in", or "founded_by"). There were some patterns used to gather the data, so it might contain some noise. 

In [ ]:
import pandas as pd

df = pd.read_pickle("relation_extraction_dataset.pkl")
df.reset_index(drop=True, inplace=True)
df.head()

,end_idx,entities,entity_spans,match,original_article,sentence,start_idx,string_id
0,1024,"[Lilium, Baillie Gifford]","[[3, 9], [151, 166]]",raising $35,Happy Friday!\n\nWe sincerely hope you and you...,"3) Lilium, a German startup that’s making an a...",1013,invested_in
1,1762,"[Facebook ’s, Giphy]","[[92, 102], [148, 153]]",acquisition,Happy Friday!\n\nWe sincerely hope you and you...,"Meanwhile, the UK’s watchdog on Friday announc...",1751,acquired_by
2,2784,"[Global-e, Vitruvian Partners]","[[27, 35], [94, 112]]",raised $60,Happy Friday!\n\nWe sincerely hope you and you...,Israeli e-commerce startup Global-e has raised...,2774,invested_in
3,680,"[Joris Van Der Gucht, Silverfin]","[[0, 19], [35, 44]]",founder,Hg is a leading investor in tax and accounting...,"Joris Van Der Gucht, co-founder at Silverfin c...",673,founded_by
4,2070,"[Tim Vandecasteele, Silverfin]","[[0, 17], [71, 80]]",founder,Hg is a leading investor in tax and accounting...,"Tim Vandecasteele, co-founder added: ""We want ...",2063,founded_by


Let's create 2 dictionaries, one that maps each label to a unique integer, and one that does it the other way around.

In [ ]:
id2label = dict()
for idx, label in enumerate(df.string_id.value_counts().index):
  id2label[idx] = label

As we can see, there are 7 labels (7 unique relationships):

In [ ]:
id2label

{0: 'founded_by',
 1: 'acquired_by',
 2: 'invested_in',
 3: 'CEO_of',
 4: 'subsidiary_of',
 5: 'partners_with',
 6: 'owned_by'}

In [ ]:
label2id = {v:k for k,v in id2label.items()}
label2id

{'CEO_of': 3,
 'acquired_by': 1,
 'founded_by': 0,
 'invested_in': 2,
 'owned_by': 6,
 'partners_with': 5,
 'subsidiary_of': 4}

## Good Luck


